# Investment crowdfunding has little faith in sustainability! At least for the moment.

## Journal:

*Venture Capital*, 25:1, 91-115, 2023, [DOI: 10.1080/13691066.2022.2129510](https://doi.org/10.1080/13691066.2022.2129510)

## Authors:

Carmen Mendoza

Isabel María Parra Oller

Álvaro Rezola (@alvarorezola)

Nuria Suárez

In [1]:
# Import libraries
import pandas as pd
import locale
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from sklearn.linear_model import LogisticRegression
import math

In [2]:
# Function definition
def summary_with_stars(model):
    # get the summary table as a DataFrame
    summary_df = model.summary2().tables[1]
    
    # create a new column for stars
    summary_df['stars'] = '-'

    # add stars based on p-values
    summary_df.loc[summary_df['P>|z|'] < 0.001, 'stars'] = '***'
    summary_df.loc[(summary_df['P>|z|'] >= 0.001) & (summary_df['P>|z|'] < 0.01), 'stars'] = '**'
    summary_df.loc[(summary_df['P>|z|'] >= 0.01) & (summary_df['P>|z|'] < 0.05), 'stars'] = '*'

    # return the modified summary table
    return summary_df

In [ ]:
# Import data
stata_dataset = "/home/alvaro/Desktop/MendozaEtAl2023-VC/Data/CROWD_SUSTAINABILITY_FINAL.dta"
df = pd.read_stata(stata_dataset)
df.drop(index=range(3679,len(df)), inplace=True)
df = df[df["form_c"] ==1]
df["deadline"] = pd.to_datetime(df["deadline"])
df["datestart"] = pd.to_datetime(df["datestart"])
df["dateincorporation"] = pd.to_datetime(df["dateincorporation"])
df = df[df["deadline"] <= "2019-10-01"]
df["totalassetsmostrecent1"] += 1e-6
df["logtotalassetsmostrecent1"] = np.log(df["totalassetsmostrecent1"])
df_sust = df[df["sustainable"] == 1]
df_non_sust = df[df["sustainable"] == 0]

We found 1,853 investment crowdfunding campaings issued under the Form C exemption from May 1st, 2016 until September 10th, 2019.

### Table 1 (a): Descriptive statistics
This table shows the descriptive statistics – mean, standard deviation, 25th percentile, median, 75th percentile – of the main variables of interest.

In [4]:
df[["exito",
    "quick75relative",
    "sustainable",
    "totalassetsmostrecent1",
    "employees",
    "loglife",                     # Age as in np.log(date_diff(start, incorporation))              
    "equity",                     
    "asked",
    "lagbranches",                 # Bank branches
    "lagvcfundraising",            # VC fundraising (not sure if then I need to do the np.log())
    "loglagnum_oper_por_platf_y",  # Number of offerings per platform
    ]].describe()

,exito,quick75relative,sustainable,totalassetsmostrecent1,employees,loglife,equity,asked,lagbranches,lagvcfundraising,loglagnum_oper_por_platf_y
count,1768.000000,1768.000000,1768.000000,1.768000e+03,1768.000000,1767.000000,1768.000000,1.768000e+03,1760.000000,1581.000000,1767.000000
mean,0.346719,0.026584,0.152149,3.344962e+05,5.359729,6.102088,0.270362,7.039715e+04,3968.019318,8813.899414,67.901527
std,0.476058,0.160909,0.359267,1.761440e+06,10.308847,1.463547,0.444270,1.184509e+05,2359.036538,11128.952148,65.634995
min,0.000000,0.000000,0.000000,1.000000e-06,0.000000,0.690000,0.000000,1.000000e+03,117.000000,0.620000,2.000000
25%,0.000000,0.000000,0.000000,1.000000e-06,1.000000,5.320000,0.000000,1.000000e+04,1484.000000,90.800003,9.000000
50%,0.000000,0.000000,0.000000,2.656100e+04,3.000000,6.350000,0.000000,2.500000e+04,4260.000000,943.030029,37.000000
75%,1.000000,0.000000,0.000000,2.040790e+05,6.000000,7.145000,1.000000,7.625000e+04,6728.000000,20763.099609,110.000000
max,1.000000,1.000000,1.000000,5.812937e+07,225.000000,9.870000,1.000000,1.070000e+06,6868.000000,33174.179688,262.000000


 ### Table 1 (b): Mean differences across subsamples
 This table shows the mean values of the main variables across the two subsamples of offerings and the T-statistic for the mean differences. The T-statistics reported are obtained for the differences between the means across groups of offerings. All the variables are defined in Annex 1. ***, **, and * indicate statistical significance at 1, 5, and 10 percent, respectively.

In [5]:
data = {
    "success": (df_non_sust["exito"],df_sust["exito"]),
    "quick75relative": (df_non_sust["quick75relative"],df_sust["quick75relative"]),
    "totalassetsmostrecent1": (df_non_sust["totalassetsmostrecent1"],df_sust["totalassetsmostrecent1"]),
    "employees": (df_non_sust["employees"],df_sust["employees"]),
    "loglife": (df_non_sust["loglife"],df_sust["loglife"]),
    "equity": (df_non_sust["equity"],df_sust["equity"]),
    "asked": (df_non_sust["asked"],df_sust["asked"]),
    "loglagnum_oper_por_platf_y": (df_non_sust["loglagnum_oper_por_platf_y"],df_sust["loglagnum_oper_por_platf_y"]),
    "lagbranches": (df_non_sust["lagbranches"],df_sust["lagbranches"]),
    "lagvcfundraising": (df_non_sust["lagvcfundraising"],df_sust["lagvcfundraising"]),
}

results = {}

for key in data:
    group1 = data[key][0]
    group2 = data[key][1]

    # calculate t-statistic    
    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy="omit")
    
    # Indicate statistical significance at different levels
    if p_value < 0.01:
        significance = "***"
    elif p_value < 0.05:
        significance = "**"
    elif p_value < 0.10:
        significance = "*"
    else:
        significance = "-"
        
    # calculate mean of each variable
    mean_group1 = np.mean(group1)
    mean_group2 = np.mean(group2)

    results[key]={'non sustainable':mean_group1,
                  'sustainable':mean_group2,
                  't-statistic':t_statistic,
                  'p-value':p_value,
                  'significance':significance}
# Display results
pd.DataFrame(results).T

# print(df_results)

,non sustainable,sustainable,t-statistic,p-value,significance
success,0.341561,0.375465,-1.075572,0.282266,-
quick75relative,0.027352,0.022305,0.473556,0.635875,-
totalassetsmostrecent1,327116.59375,375619.4375,-0.415749,0.677644,-
employees,5.491661,4.624535,1.270525,0.204065,-
loglife,6.067376,6.29539,-2.355741,0.018595,**
equity,0.262175,0.315985,-1.830379,0.067362,*
asked,70083.695312,72143.882812,-0.262597,0.792892,-
loglagnum_oper_por_platf_y,68.056076,67.040894,0.23351,0.815392,-
lagbranches,4045.154722,3536.696629,3.252588,0.001165,***
lagvcfundraising,8817.849609,8790.697266,0.034194,0.972727,-


### Table 2: Propensity Score Matching 
This table shows the mean values of the main variables across the two subsamples of offerings and the t-statistics obtained for the differences between the means across groups of offerings, before matching and after matching using caliper, nearest 1-to-1 and nn-VBC methods. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

In [10]:
# Two probit functions to validate covariates
formula_traetment = f"sustainable ~ logtotalassetsmostrecent1 + logemployees1 + logasked1"
summary_with_stars(smf.probit(formula=formula_traetment, data=df).fit())

Optimization terminated successfully.
         Current function value: 0.424859
         Iterations 5


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],stars
Intercept,-0.393784,0.345630,-1.139322,0.254569,-1.071208,0.283639,-
logtotalassetsmostrecent1,0.004709,0.003481,1.352971,0.176065,-0.002113,0.011532,-
logemployees1,-0.077350,0.046525,-1.662531,0.096406,-0.168537,0.013838,-
logasked1,-0.052549,0.032199,-1.632018,0.102676,-0.115657,0.010559,-


In [6]:
formula_outcome = f"exito ~ logtotalassetsmostrecent1 + logemployees1 + logasked1"
summary_with_stars(smf.probit(formula=formula_outcome, data=df).fit())

Optimization terminated successfully.
         Current function value: 0.625983
         Iterations 5


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],stars
Intercept,1.504206,0.304060,4.947075,7.533708e-07,0.908260,2.100152,***
logtotalassetsmostrecent1,0.003500,0.002956,1.184039,2.363977e-01,-0.002294,0.009295,-
logemployees1,0.127701,0.039074,3.268154,1.082515e-03,0.051117,0.204285,**
logasked1,-0.202604,0.028695,-7.060672,1.656997e-12,-0.258845,-0.146363,***


### Table 2 (A): Before Matching

In [7]:
df[["exito", "quick75relative", "logtotalassetsmostrecent1", "logemployees1", "logasked1"]].describe()
df_sust = df[df["sustainable"] == 1]
df_non_sust = df[df["sustainable"] == 0]

data = {
    "success": (df_sust["exito"], df_non_sust["exito"]),
    "quick75relative": (df_sust["quick75relative"], df_non_sust["quick75relative"]),
    "logtotalassetsmostrecent1": (df_sust["logtotalassetsmostrecent1"], df_non_sust["logtotalassetsmostrecent1"]),
    "logemployees1": (df_sust["logemployees1"], df_non_sust["logemployees1"]),
    "logasked1": (df_sust["logasked1"], df_non_sust["logasked1"])
}

results = {}

for key in data:
    group1 = data[key][0]
    group2 = data[key][1]

    # calculate t-statistic    
    t_statistic, p_value = stats.ttest_ind(group1, group2, nan_policy="omit")
    
    # Indicate statistical significance at different levels
    if p_value < 0.01:
        significance = "***"
    elif p_value < 0.05:
        significance = "**"
    elif p_value < 0.10:
        significance = "*"
    else:
        significance = "-"
        
    # calculate mean of each variable
    mean_group1 = np.mean(group1)
    mean_group2 = np.mean(group2)

    results[key]={'sustainable':mean_group1,
                  'non_sustainable':mean_group2,
                  "difference" :(mean_group1 - mean_group2),
                  't-statistic':t_statistic,
                  'p-value':p_value,
                  'significance':significance}
# Display results
pd.DataFrame(results).T

,sustainable,non_sustainable,difference,t-statistic,p-value,significance
success,0.375465,0.341561,0.033904,1.075572,0.282266,-
quick75relative,0.022305,0.027352,-0.005047,-0.473556,0.635875,-
logtotalassetsmostrecent1,4.746425,4.217429,0.528996,0.710444,0.477523,-
logemployees1,1.367925,1.43493,-0.067005,-1.199181,0.230619,-
logasked1,10.325472,10.433107,-0.107635,-1.4167,0.156747,-


### Table 2 (B): After Matching

In [8]:
# Logit model para estimar el puntaje de propensión (ps)
model = LogisticRegression()
df = df.dropna(axis = 1) # drop all variables that have empty values

# Independent variables
X = df[["logtotalassetsmostrecent1",
        "logemployees1",
        "logasked1"]]

# Dependent variable (treatment group)
y = df["sustainable"]

# Model adjustment & predicted probabilities
model.fit(X, y) 
pred_prob = model.predict_proba(X)
df["ps"] = pred_prob[:, 1]

# calculamos el logit del puntuaje de propensión para emparejar
def logit(p):
    logit_value = math.log(p/(1 - p))
    return logit_value

df["ps_logit"] = df.ps.apply(logit)

# calcular standard errors ATE
def ate_se(df):
    # obtener el numero de observaciones de cada grupo
    n_t = df[df.sustainable == 1].shape[0]
    n_c = df[df.sustainable == 0].shape[0]
    
    # obtener la varianza de los resultados en cada grupo
    s_t = df[df.sustainable == 1].exito.var()
    s_c = df[df.sustainable == 0].exito.var()
    
    # calcular error estandar del ATE usando la fórmula
    se = np.sqrt((s_t / n_t) + (s_c / n_c))
    
    return se

# Implementing the caliper match
def caliper_match(df, threshold):
    # ordenar los datos por ps_logit y crear una columna con el índice original
    df_sorted = df.sort_values("ps_logit").reset_index()
    df_sorted["orig_index"] = df_sorted.index
    
    # crear listas vacias para almacenar los indices emparejados y no emparejados
    matched_index = []
    unmatched_index = []
    
    # iterar sobre las filas del dataframe ordenado
    for i in range(len(df_sorted)):
        row = df_sorted.iloc[i]
        if i not in matched_index: # si la fila no está emparejada todavía
            potential_matches = df_sorted[(df_sorted.sustainable != row.sustainable) & (abs(df_sorted.ps_logit - row.ps_logit) <= threshold)]
            
            # encontrar las filas potenciales que tienen un tratamiento diferente y una diferencia de ps_logit menor o igual al umbral
            
            if len(potential_matches) > 0: # si hay al menos una fila potencialmente emparejable
                closest_match_index = potential_matches.iloc[0].orig_index
                # Tomar la priemra fila potencial como la más cercana
                
                matched_index.append(i)
                matched_index.append(closest_match_index)
                # añadir ambos índices a la lista de emparejados
                
            else:
                unmatched_index.append(i)
                # si no hay ninguna fila potencialmente emparejable, añadir el índice a la lista d eno emparejados
    return matched_index, unmatched_index

caliper_matched, caliper_unmatched = caliper_match(df[["sustainable", "logtotalassetsmostrecent1", "logemployees1", "logasked1", "ps_logit"]], 0.1) # umbral 0.2
caliper_df = df.iloc[caliper_matched]

### Table 3: Average Treatment Effect on the Treated (ATET) 
This table shows the average treatment effect on the treated individuals (ATET) for each method: caliper, nearest 1-to-1 and nn-VBC methods. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

In [9]:
caliper_ate_success = caliper_df.groupby("sustainable")["exito"].mean().diff().iloc[-1]
caliper_ate_quick = caliper_df.groupby("sustainable")["quick75relative"].mean().diff().iloc[-1]
caliper_se = ate_se(caliper_df)

# realizar una preuba de t de dos muestras independientes y obtener los valores p y los intervalso de confianza al 95%
caliper_success_p, caliper_success_ci, caliper_success_df = sms.ttest_ind(caliper_df[caliper_df.sustainable == 1].exito.values,
                                                  caliper_df[caliper_df.sustainable == 0].exito.values,
                                                  usevar="unequal",
                                                  alternative="larger", value=0)

# realizar una preuba de t de dos muestras independientes y obtener los valores p y los intervalso de confianza al 95%
caliper_quick_p, caliper_quick_ci, caliper_quick_df = sms.ttest_ind(caliper_df[caliper_df.sustainable == 1].quick75relative.values,
                                                  caliper_df[caliper_df.sustainable == 0].quick75relative.values,
                                                  usevar="unequal",
                                                  alternative="larger", value=0)

data = {"Modelo caliper 1-to-1": ["success", "quickrelative75"],
        "ATE": [caliper_ate_success, caliper_ate_quick],
        "Error estándar": [caliper_se, caliper_se],
        "Valor p": [caliper_success_p, caliper_quick_p],
        "Intervalo de confianza (95%)": [caliper_success_ci, caliper_quick_ci]}

pd.DataFrame(data)

,Modelo caliper 1-to-1,ATE,Error estándar,Valor p,Intervalo de confianza (95%)
0,success,0.064235,0.021758,2.952188,0.001622
1,quickrelative75,-0.010451,0.021758,-1.724071,0.957501


### Table 4: Sustainability and success
This table presents IV results examining the effect of the sustainable orientation of investment crowdfunding offerings on the probability of success. The dependent variable in columns (1) and (2) is the dummy that identifies sustainable offerings (Sustainable). The dependent variable in columns (3) and (5) is SUCCESS. QUICK75 is the dependent variables in columns (4) and (6). Variables definitions are reported in Annex 1. Year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***, ** and * indicate statistical significance at 1, 5, and 10%, respectively.

### Table 5: Sustainability and success: the role of firm- and offering-level characteristics
This table presents results examining the effect of firm- and offering-level characteristics on the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. *** and ** indicate statistical significance at 1 and 5 percent, respectively.

### Table 6: Sustainability and success: the role of the financing environment
This table presents results examining the effect of the characteristics of the financing environment on the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Firm and offering control variables, year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***; ** and * indicate statistical significance at 1, 5, and 10 percent, respectively.

### Table 7: Sustainability and success: robustness tests
This table presents a set of robustness tests for the relationship between the sustainable orientation of investment crowdfunding offerings and the probability of success. In column (1), we report the results for the second-stage regression for the Heckman (1979) method. In column (2), we find that the results do not vary when controlling for the characteristics of the team in terms of gender and size. In columns (3) to (5), we control for the funding history of the company. In column (6), we control for the cost structure defined by the funding portal. The dependent variable is SUCCESS. Variables definitions are reported in Annex 1. Firm and offering control variables, year, industry-year and state-year fixed effects are included but not reported. T-statistics are in parentheses. ***; ** and * indicate statistical significance at 1, 5, and 10 percent, respectively.